In [1]:
import requests  # Импортирует библиотеку для выполнения HTTP-запросов.
from bs4 import BeautifulSoup  # Импортирует библиотеку для парсинга HTML и XML.

# Открывает файл quotes.txt для записи, с кодировкой UTF-8.
with open('quotes.txt', 'w', encoding='utf-8') as f:
    k = 1  # Инициализирует счетчик для нумерации цитат.

    # Словари для хранения информации об авторах, цитатах и тегах.
    tags_dict = {}  # Для хранения тегов и связанных с ними цитат.
    authors_url_dict = {}  # Для хранения URL авторов.
    authors_quote_dict = {}  # Для хранения цитат по авторам.

    # Итерируется по 10 страницам сайта.
    for p in range(1, 11):
        url = f'https://quotes.toscrape.com/page/{p}'  # Формирует URL страницы.
        response = requests.get(url)  # Выполняет GET-запрос к URL.

        soup = BeautifulSoup(response.text, 'html.parser')  # Парсит HTML страницы.
        quotes = soup.find_all('span', class_='text')  # Находит все цитаты.
        authors = soup.find_all('small', class_='author')  # Находит всех авторов.
        tags = soup.find_all('div', class_='tags')  # Находит все теги для цитат.
        divs = soup.find_all('div', class_='quote')  # Находит все блоки с цитатами.

        # Итерируется по всем найденным цитатам.
        for i in range(len(quotes)):
            if authors_url_dict.get(authors[i].text, 0) == 0:  # Проверяет, есть ли уже URL автора.
                authors_url_dict[authors[i].text] = 'https://quotes.toscrape.com' + divs[i].a['href']  # Сохраняет URL автора.
            if authors_quote_dict.get(authors[i].text, 0) == 0:  # Проверяет, есть ли уже цитаты автора.
                authors_quote_dict[authors[i].text] = []  # Инициализирует список цитат для автора.
                authors_quote_dict[authors[i].text].append(quotes[i].text)  # Добавляет цитату в список.
            else:
                authors_quote_dict[authors[i].text].append(quotes[i].text)  # Добавляет цитату к существующим цитатам.

            # Записывает цитату, автора и теги в файл.
            f.write(f'{k}) Quote: {quotes[i].text}\n')  # Записывает номер и текст цитаты.
            f.write(len(f'{k}) ') * ' ' + f'Author: {authors[i].text}\n')  # Записывает имя автора.
            my_tags = tags[i].find_all('a', class_='tag')  # Извлекает теги для текущей цитаты.
            f.write(len(f'{k}) ') * ' ' + f'Tags: ')  # Записывает заголовок для тегов.

            # Итерируется по каждому тегу.
            for my_tag in my_tags:
                f.write(my_tag.text)  # Записывает текст тега.
                if tags_dict.get(my_tag.text, 0) == 0:  # Проверяет, есть ли уже тег в словаре.
                    tags_dict[my_tag.text] = []  # Инициализирует список цитат для тега.
                    tags_dict[my_tag.text].append(quotes[i].text)  # Добавляет цитату в список тегов.
                else:
                    tags_dict[my_tag.text].append(quotes[i].text)  # Добавляет цитату к существующим.

            f.write('\n\n')  # Печатает пустую строку для разделения записей.
            k += 1  # Увеличивает счетчик цитат.

# Открывает файл tags.txt для записи, с кодировкой UTF-8.
with open('tags.txt', 'w', encoding='utf-8') as f:
    k = 1  # Инициализирует счетчик для тегов.

    # Итерируется по каждому тегу в словаре.
    for tag in tags_dict:
        f.write(f'{k}) Tag: {tag}\n')  # Записывает номер и текст тега.
        f.write(len(f'{k}) ') * ' ' + f'Quotes: \n')  # Записывает заголовок для цитат.
        n = 1  # Инициализирует счетчик для цитат под тегом.

        # Итерируется по каждой цитате для текущего тега.
        for quote in tags_dict[tag]:
            f.write(len(f'{k}) ') * ' ' + f'{n}. {quote}\n')  # Записывает номер цитаты и текст.
            n += 1  # Увеличивает счетчик цитат.
        f.write('\n')  # Печатает пустую строку для разделения записей.
        k += 1  # Увеличивает счетчик тегов.

# Открывает файл authors.txt для записи, с кодировкой UTF-8.
with open('authors.txt', 'w', encoding='utf-8') as f:
    k = 1  # Инициализирует счетчик для авторов.

    # Итерируется по каждому автору в словаре цитат.
    for author in authors_quote_dict:
        response = requests.get(authors_url_dict[author])  # Выполняет GET-запрос по URL автора.
        soup = BeautifulSoup(response.text, 'html.parser')  # Парсит HTML страницы автора.

        # Извлекает информацию об авторе.
        born_date = soup.find('span', class_='author-born-date')  # Находит дату рождения автора.
        born_location = soup.find('span', class_='author-born-location')  # Находит место рождения автора.
        author_description = soup.find('div', class_='author-description')  # Находит описание автора.

        # Записывает информацию об авторе и его цитаты в файл.
        f.write(f'{k}) Author: {author}\n')  # Записывает номер и имя автора.
        f.write(len(f'{k}) ') * ' ' + f'Born date: {born_date.text}\n')  # Записывает дату рождения.
        f.write(len(f'{k}) ') * ' ' + f'Born location: {born_location.text}\n')  # Записывает место рождения.
        f.write(len(f'{k}) ') * ' ' + f'Description: {author_description.text.strip()}\n')  # Записывает описание автора.
        f.write(len(f'{k}) ') * ' ' + f'Quotes: \n')  # Записывает заголовок для цитат автора.

        n = 1  # Инициализирует счетчик для цитат под автором.

        # Итерируется по каждой цитате для текущего автора.
        for quote in authors_quote_dict[author]:
            f.write(len(f'{k}) ') * ' ' + f'{n}. {quote}\n')  # Записывает номер цитаты и текст.
            n += 1  # Увеличивает счетчик цитат.
        f.write('\n')  # Печатает пустую строку для разделения записей.
        k += 1  # Увеличивает счетчик авторов.